# Import thư viện

In [ ]:
# Import thư viện 
from __future__ import annotations

from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

RANDOM_STATE: int = 42
np.random.seed(RANDOM_STATE)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

# Xác định project root để chạy ổn định (dù mở notebook từ thư mục nào)
project_root = Path.cwd()
if project_root.name.lower() == 'notebooks':
    project_root = project_root.parent

data_dir = project_root / 'data' / 'preprocessed_data'
print('Project root:', project_root)
print('Preprocessed dir:', data_dir)

Project root: d:\Vscode\Self\Data Science\Book Notes\IntroDS\Final Project github\Intro2DS
Preprocessed dir: d:\Vscode\Self\Data Science\Book Notes\IntroDS\Final Project github\Intro2DS\data\preprocessed_data




> Quy ước: dữ liệu cho từng giả thuyết đã được chuẩn bị và lưu từ notebook `04_feature_engineering_Thanh.ipynb` vào `data/preprocessed_data/`:
- Với mỗi giả thuyết Hx: lưu `X_train_Hx_Thanh.csv`, `X_test_Hx_Thanh.csv`.
- Riêng nhãn (dùng chung cho mọi giả thuyết): lưu một lần từ H1 là `y_train_H1_Thanh.csv`, `y_test_H1_Thanh.csv`.

## Format báo cáo kết quả (theo từng mô hình)
**Linear Regression**
- H1: fit → tính metrics → (nhóm sẽ điền nhận xét sau khi có output)
- H2: tương tự
- …
- Hx

**Từ kết quả baseline:** so sánh mức độ phù hợp giữa các giả thuyết → chọn giả thuyết tốt nhất → giải thích ngắn gọn (dựa vào ngữ nghĩa thực tế).

**XGBoost**
Test Hypothesis tốt nhất từ Linear Regression để xem kết quả.

**Random Forest**
Test Hypothesis tốt nhất từ Linear Regression để xem kết quả.


# Baseline Linear Regression

In [ ]:
# Linear Regression - Baseline (y dùng chung từ H1)

def load_x_split(h: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load X_train/X_test cho giả thuyết H{h}.

    Args:
        h: Số giả thuyết dạng chuỗi, ví dụ '1' cho H1, '134' cho H134.

    Returns:
        (X_train, X_test)
    """
    x_train_path = data_dir / f"X_train_H{h}_Thanh.csv"
    x_test_path = data_dir / f"X_test_H{h}_Thanh.csv"

    for p in [x_train_path, x_test_path]:
        if not p.exists():
            raise FileNotFoundError(f"Thiếu file: {p}.")

    X_train = pd.read_csv(x_train_path)
    X_test = pd.read_csv(x_test_path)
    return X_train, X_test


def load_shared_y_from_h1() -> Tuple[pd.Series, pd.Series]:
    """Load y_train/y_test dùng chung từ H1 (vote_average)."""
    y_train_path = data_dir / "y_train_H1_Thanh.csv"
    y_test_path = data_dir / "y_test_H1_Thanh.csv"

    for p in [y_train_path, y_test_path]:
        if not p.exists():
            raise FileNotFoundError(f"Thiếu file y (H1): {p}.")

    y_train_df = pd.read_csv(y_train_path)
    y_test_df = pd.read_csv(y_test_path)
    if y_train_df.shape[1] != 1 or y_test_df.shape[1] != 1:
        raise ValueError("y_train/y_test phải có đúng 1 cột (vote_average).")

    y_train = y_train_df.iloc[:, 0]
    y_test = y_test_df.iloc[:, 0]
    return y_train, y_test


def regression_metrics(y_true: pd.Series, y_pred: np.ndarray) -> Dict[str, float]:
    """Tính metrics cho bài toán hồi quy."""
    mae = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    r2 = float(r2_score(y_true, y_pred))
    return {"mae": mae, "rmse": rmse, "r2": r2}


# Load y dùng chung một lần (H1)
y_train, y_test = load_shared_y_from_h1()

# Chạy Linear Regression cho từng giả thuyết (chỉ khác X; y dùng chung)
results: List[Dict[str, object]] = []

# Nếu chưa tạo đủ file X, notebook sẽ tự skip và in cảnh báo thay vì dừng lỗi.
hypotheses_to_try = ["1", "2", "3", "4", "134"]

for h in hypotheses_to_try:
    try:
        X_train, X_test = load_x_split(h)
    except FileNotFoundError as e:
        print(f"\n[SKIP] H{h}: {e}")
        continue

    if len(X_train) != len(y_train) or len(X_test) != len(y_test):
        raise ValueError(
            f"Kích thước X/y không khớp cho H{h}: "
            f"X_train={len(X_train)}, y_train={len(y_train)}, "
            f"X_test={len(X_test)}, y_test={len(y_test)}"
        )

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    m = regression_metrics(y_test, y_pred)

    results.append({
        "model": "LinearRegression",
        "hypothesis": f"H{h}",
        **m,
    })

    print(f"\nLinear Regression - H{h}")
    print("MAE :", round(m["mae"], 4))
    print("RMSE:", round(m["rmse"], 4))
    print("R^2 :", round(m["r2"], 4))

# Bảng tổng kết baseline (Linear Regression)
baseline_df = pd.DataFrame(results).sort_values(["model", "hypothesis"]).reset_index(drop=True)
display(baseline_df)


Linear Regression - H1
MAE : 1.0646
RMSE: 1.4699
R^2 : 0.0297

Linear Regression - H2
MAE : 1.0806
RMSE: 1.4792
R^2 : 0.0173

Linear Regression - H3
MAE : 1.064
RMSE: 1.4629
R^2 : 0.0389

Linear Regression - H4
MAE : 0.9838
RMSE: 1.3919
R^2 : 0.1299

Linear Regression - H134
MAE : 0.9754
RMSE: 1.3812
R^2 : 0.1432


,model,hypothesis,mae,rmse,r2
0,LinearRegression,H1,1.064593,1.469856,0.029727
1,LinearRegression,H134,0.975379,1.381218,0.143222
2,LinearRegression,H2,1.080573,1.479222,0.017323
3,LinearRegression,H3,1.063961,1.462896,0.038894
4,LinearRegression,H4,0.983775,1.391906,0.129911


Dựa trên baseline Linear Regression, H134 cho kết quả tốt nhất trong nhóm giả thuyết: việc kết hợp biến thời gian (`movie_age`) và nhóm biến phân loại (genres/language/certification) giúp mô hình tuyến tính nắm bắt thêm thông tin ngoài các biến số cơ bản của H1.

Tuy nhiên, mức $R^2$ vẫn còn thấp → quan hệ giữa đặc trưng và `vote_average` có xu hướng phi tuyến/đa tương tác; vì vậy các mô hình cây tăng cường (XGBoost/Random Forest) là lựa chọn hợp lý để cải thiện hiệu năng trên cùng tập $X$ đã tạo.

# XGBoost

In [ ]:
# XGBoost (H134) - Hyperparameter tuning bằng K-Fold trên tập train

# Fallback helpers nếu chạy cell này độc lập
if "load_x_split" not in globals():
    def load_x_split(h: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        x_train_path = data_dir / f"X_train_H{h}_Thanh.csv"
        x_test_path = data_dir / f"X_test_H{h}_Thanh.csv"
        for p in [x_train_path, x_test_path]:
            if not p.exists():
                raise FileNotFoundError(f"Thiếu file: {p}.")
        return pd.read_csv(x_train_path), pd.read_csv(x_test_path)

if "load_shared_y_from_h1" not in globals():
    def load_shared_y_from_h1() -> Tuple[pd.Series, pd.Series]:
        y_train_path = data_dir / "y_train_H1_Thanh.csv"
        y_test_path = data_dir / "y_test_H1_Thanh.csv"
        for p in [y_train_path, y_test_path]:
            if not p.exists():
                raise FileNotFoundError(f"Thiếu file y (H1): {p}.")
        y_train_df = pd.read_csv(y_train_path)
        y_test_df = pd.read_csv(y_test_path)
        return y_train_df.iloc[:, 0], y_test_df.iloc[:, 0]

if "regression_metrics" not in globals():
    def regression_metrics(y_true: pd.Series, y_pred: np.ndarray) -> Dict[str, float]:
        mae = float(mean_absolute_error(y_true, y_pred))
        rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
        r2 = float(r2_score(y_true, y_pred))
        return {"mae": mae, "rmse": rmse, "r2": r2}

# Load dữ liệu
h = "134"
X_train, X_test = load_x_split(h)
y_train, y_test = load_shared_y_from_h1()

if len(X_train) != len(y_train) or len(X_test) != len(y_test):
    raise ValueError(
        f"Kích thước X/y không khớp cho H{h}: "
        f"X_train={len(X_train)}, y_train={len(y_train)}, "
        f"X_test={len(X_test)}, y_test={len(y_test)}"
    )

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    random_state=RANDOM_STATE,
    n_jobs=-1,
 )

param_distributions = {
    "n_estimators": [300, 500, 800],
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.03, 0.05, 0.1],
    "subsample": [0.7, 0.85, 1.0],
    "colsample_bytree": [0.7, 0.85, 1.0],
    "min_child_weight": [1, 5, 10],
    "reg_alpha": [0.0, 0.1, 1.0],
    "reg_lambda": [1.0, 5.0, 10.0],
}

search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=25,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0,
 )

search.fit(X_train, y_train)

best_rmse_cv = float(-search.best_score_)
print("XGBoost (H134) - Best CV RMSE:", round(best_rmse_cv, 4))
print("XGBoost (H134) - Best params:")
print(search.best_params_)

best_model = search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
m = regression_metrics(y_test, y_pred)

print("\nXGBoost - H134 (test)")
print("MAE :", round(m["mae"], 4))
print("RMSE:", round(m["rmse"], 4))
print("R^2 :", round(m["r2"], 4))

XGBoost (H134) - Best CV RMSE: 1.349
XGBoost (H134) - Best params:
{'subsample': 1.0, 'reg_lambda': 5.0, 'reg_alpha': 1.0, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.03, 'colsample_bytree': 0.7}

XGBoost - H134 (test)
MAE : 0.9003
RMSE: 1.3067
R^2 : 0.2332


# Random Forest

In [ ]:
# Random Forest (H134) - Hyperparameter tuning bằng K-Fold trên tập train

# Fallback helpers nếu chạy cell này độc lập
if "load_x_split" not in globals():
    def load_x_split(h: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        x_train_path = data_dir / f"X_train_H{h}_Thanh.csv"
        x_test_path = data_dir / f"X_test_H{h}_Thanh.csv"
        for p in [x_train_path, x_test_path]:
            if not p.exists():
                raise FileNotFoundError(f"Thiếu file: {p}.")
        return pd.read_csv(x_train_path), pd.read_csv(x_test_path)

if "load_shared_y_from_h1" not in globals():
    def load_shared_y_from_h1() -> Tuple[pd.Series, pd.Series]:
        y_train_path = data_dir / "y_train_H1_Thanh.csv"
        y_test_path = data_dir / "y_test_H1_Thanh.csv"
        for p in [y_train_path, y_test_path]:
            if not p.exists():
                raise FileNotFoundError(f"Thiếu file y (H1): {p}.")
        y_train_df = pd.read_csv(y_train_path)
        y_test_df = pd.read_csv(y_test_path)
        return y_train_df.iloc[:, 0], y_test_df.iloc[:, 0]

if "regression_metrics" not in globals():
    def regression_metrics(y_true: pd.Series, y_pred: np.ndarray) -> Dict[str, float]:
        mae = float(mean_absolute_error(y_true, y_pred))
        rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
        r2 = float(r2_score(y_true, y_pred))
        return {"mae": mae, "rmse": rmse, "r2": r2}

# Load dữ liệu
h = "134"
X_train, X_test = load_x_split(h)
y_train, y_test = load_shared_y_from_h1()

if len(X_train) != len(y_train) or len(X_test) != len(y_test):
    raise ValueError(
        f"Kích thước X/y không khớp cho H{h}: "
        f"X_train={len(X_train)}, y_train={len(y_train)}, "
        f"X_test={len(X_test)}, y_test={len(y_test)}"
    )

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

rf = RandomForestRegressor(
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

param_distributions = {
    "n_estimators": [300, 600, 1000],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", 0.5, 1.0],
    "bootstrap": [True],
}

search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=25,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0,
 )

search.fit(X_train, y_train)

best_rmse_cv = float(-search.best_score_)
print("Random Forest (H134) - Best CV RMSE:", round(best_rmse_cv, 4))
print("Random Forest (H134) - Best params:")
print(search.best_params_)

best_model = search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
m = regression_metrics(y_test, y_pred)

print("\nRandom Forest - H134 (test)")
print("MAE :", round(m["mae"], 4))
print("RMSE:", round(m["rmse"], 4))
print("R^2 :", round(m["r2"], 4))

Random Forest (H134) - Best CV RMSE: 1.3449
Random Forest (H134) - Best params:
{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}

Random Forest - H134 (test)
MAE : 0.902
RMSE: 1.3077
R^2 : 0.232
